In [30]:
import sys
sys.path.insert(0, '/home/johnr_000/jupyter_blog/jupyter_blog/scripts')
import eia_model as em
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from datetime import datetime

In [19]:
#variables to specify date range and frequency
api_key = em.eia_api
start = '2015-12-01 01:00:00'
end = '2016-01-31 23:00:00'
freq = 'H'
# create list of series used for dictionary keys
keys = ['EBA.BPAT-ALL.D.H', 'EBA.PACE-ALL.D.H', 'EBA.PACW-ALL.D.H', 
        'EBA.PGE-ALL.D.H', 'EBA.PSEI-ALL.D.H', 'EBA.SCL-ALL.D.H']
# create dict of GetSeriesRange objects from series list
series_dict = {key: em.GetSeries(api_key=api_key, series_id=key, 
                                   start=start, end=end, freq=freq) for key in keys}

In [20]:
# loop over dictionary and print out full name of each series
for key in series_dict:
    print('{}: {}'.format(series_dict[key].data.series[0]['name'],key))

Demand for PacifiCorp West (PACW), Hourly: EBA.PACW-ALL.D.H
Demand for Seattle City Light (SCL), Hourly: EBA.SCL-ALL.D.H
Demand for Portland General Electric Company (PGE), Hourly: EBA.PGE-ALL.D.H
Demand for Puget Sound Energy, Inc. (PSEI), Hourly: EBA.PSEI-ALL.D.H
Demand for Bonneville Power Administration (BPAT), Hourly: EBA.BPAT-ALL.D.H
Demand for PacifiCorp East (PACE), Hourly: EBA.PACE-ALL.D.H


In [21]:
# all the dataframes have the same date range so we can concatenate in a loop
concat_df = pd.concat([series_dict[key].data.df for key in series_dict], axis=1)
# and output as a pickle file - I'm doing this to avoid calling the EIA API as I develop
concat_df.to_pickle('data/nw_load_df.pkl')
# concat_df = pd.read_pickle('data/new_load_df.pkl')

In [12]:
concat_df = pd.read_pickle('/home/johnr_000/jupyter_blog/jupyter_blog/data/nw_load_df.pkl')

In [23]:
concat_df.describe()

/home/johnr_000/venv_python3/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,EBA.PACW-ALL.D.H,EBA.SCL-ALL.D.H,EBA.PGE-ALL.D.H,EBA.PSEI-ALL.D.H,EBA.BPAT-ALL.D.H,EBA.PACE-ALL.D.H
count,1.487000e+03,1487.000000,1487.000000,1482.000000,1487.000000,1.487000e+03
mean,4.451198e+03,1282.258911,2579.921991,3946.066127,7059.477471,-2.470617e+04
std,6.558528e+04,177.394473,385.764476,554.268601,833.190306,8.950455e+05
min,-6.785000e+03,868.000000,1730.000000,2292.000000,5006.000000,-3.199017e+07
25%,2.393500e+03,1128.500000,2272.000000,NaN,6451.500000,5.169000e+03
50%,2.709000e+03,1324.000000,2646.000000,NaN,7109.000000,5.508000e+03
75%,2.941500e+03,1416.000000,2872.500000,NaN,7637.500000,5.822500e+03
max,2.530003e+06,1654.000000,3472.000000,5088.000000,9365.000000,5.966800e+04


In [104]:
train, test = train_test_split(concat_df, test_size = 0.2)
print('Length of training dataframe: {} and testing dataframe {}'.format(len(train), len(test)))

Length of training dataframe: 1189 and testing dataframe 298


In [107]:
rng = np.random.RandomState(42)
clf = IsolationForest(max_samples=100, random_state=rng)
train_x = train.index.to_datetime().hour
train_y =  train['EBA.SCL-ALL.D.H'].values
test_x = test.index.to_datetime().hour
test_y =  test['EBA.SCL-ALL.D.H'].values
clf.fit(np.r_[train_x.reshape(len(train_x), 1), train_y])
y_pred = clf.predict(np.r_[test_x.reshape(len(test_x), 1), test_y])

ValueError: all the input arrays must have same number of dimensions

In [108]:
# Generate train data
X = 0.3 * rng.randn(100, 2)
X_train = np.r_[X + 2, X - 2]

In [136]:
x1 = train.index.to_datetime().hour
y1 = train['EBA.SCL-ALL.D.H'].values
test_ar = np.swapaxes(np.array([x1, y1]), 0, 1)
test_ar.shape

(1189, 2)

In [137]:
X_train.shape

(200, 2)